In [1]:
import torch
import os
import pandas as pd
import numpy as np
from torch import nn
from torch.nn import functional as F
from sklearn.preprocessing import OneHotEncoder

from torch.utils.data import DataLoader, Dataset

In [7]:
df = pd.read_csv('dataset.csv').dropna()
max_len = len(max(df.text,key = lambda a:len(a)))
print('max_len',max_len)
print('len',len(df))
print('col',df.columns)

max_len 1903
len 39797
col Index(['text', 'tag'], dtype='object')


In [8]:
len(df)

39797

In [9]:
ALPHABET = " abcdefghijklmnopqrstuvwxyz"
num_feat = len(ALPHABET)
LETTER_POS_DICT = {char: i for i, char in enumerate(ALPHABET)}
POS_LETTER_DICT = {i: char for i, char in enumerate(ALPHABET)}

In [10]:
inp =  np.zeros((len(df),max_len,num_feat),np.float32)

for i,row in df.iterrows():
    for j,c in enumerate(row.text):
        inp[i][j][LETTER_POS_DICT[c]] = 1
    if i%1000 == 0:
        print(i)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000


In [11]:
inp.shape

(39797, 1903, 27)

In [12]:
out = df.tag.values
ohe = OneHotEncoder()

In [13]:
out.shape

(39797,)

In [14]:
out_exp = ohe.fit_transform(out.reshape(-1,1))

In [15]:
inp[0].shape

(1903, 27)

In [16]:
x = torch.from_numpy(inp)
y = torch.from_numpy(out)

In [17]:
class MyDataset(Dataset):
    def __init__(self, x,y, n_inp):
        self.x , self.y = (torch.from_numpy(x),
                           torch.from_numpy(y))
        
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
    
    def __len__(self):
        return len(self.x)

myData = MyDataset(inp,out_exp.toarray(), 1)

data_loader = DataLoader(myData, batch_size=4, shuffle =True,drop_last=True)
train_size = int(0.8 * len(data_loader))
test_size = len(data_loader) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(data_loader, [train_size, test_size])

In [18]:
for data in data_loader:
    print(data)
    break

[tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 1.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0.,

In [19]:
a_inp,a_out = data

In [20]:
embed_size = 300
hidden_layers = 1
hidden_size = 64
output_size = 6
max_epochs = 15
hidden_size_linear = 64
lr = 0.5
batch_size = 128
seq_len = max_len # Sequence length for RNN
dropout_keep = 0.8
input_size = num_feat

In [25]:
# input shape = (batch size, sequence length, num features) | N,L,Hin
lstm_layer = nn.LSTM(num_feat, hidden_size, 5,batch_first=True,bidirectional = True)
# output shape = (batch size, sequence length, 2 * hidden_size)

# concatenate over num features and hidden size
# output shape = (batch size, seq_len, 2 * hidden_size + num features)

# linear layer
linear_layer = nn.Linear(2 * hidden_size + num_feat,hidden_size_linear)
# output size = (batch_size, seq_len, hidden_size_linear)

# tanh - no change in dims
tanh_layer = nn.Tanh()

# dropout - no change of dimensions
dropout_layer = nn.Dropout(dropout_keep)

# fc layer - same operation as linear layer
fc_layer = nn.Linear(hidden_size_linear,output_size)
# output shape - (batch_size, hidden_size_linear)

# softmax - no change
softmax_layer = nn.Softmax(dim=1)


In [26]:
# input shape = (batch size, sequence length, num features) | N,L,Hin
lstm_output,_ = lstm_layer(a_inp)
# output shape = (batch size, sequence length, 2 * hidden_size)

# concatenate over num features and hidden size
input_features = torch.cat([lstm_output,a_inp],2)
# output shape = (batch size, sequence length, 2 * hidden_size + num features)

# linear output
linear_output = linear_layer(input_features)
# output size = (batch_size, seq_len, hidden_size_linear)

linear_output = linear_output.permute(0,2,1) # Reshaping fot max_pool
        
max_out_features = F.max_pool1d(linear_output, linear_output.shape[2]).squeeze(2)
# max_out_features.shape = (batch_size, hidden_size_linear)

max_out_features = dropout_layer(max_out_features)
final_out = fc_layer(max_out_features)
res = softmax_layer(final_out)


In [33]:
loss = torch.nn.CrossEntropyLoss()

loss(res,a_out)

tensor(1.7997, dtype=torch.float64, grad_fn=<DivBackward1>)

In [34]:
res.shape

torch.Size([4, 6])

In [30]:
a_out

tensor([[0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0.]], dtype=torch.float64)

In [180]:
a_out.argmax(axis=0)

tensor([2, 0, 0, 0, 1, 3])

In [171]:
class RCNN(nn.Module):
    def __init__(self):
        super(RCNN, self).__init__()
        # input shape = (batch size, sequence length, num features) | N,L,Hin
        self.lstm_layer = nn.LSTM(num_feat, hidden_size, 5,batch_first=True,bidirectional = True)
        # output shape = (batch size, sequence length, 2 * hidden_size)

        # concatenate over num features and hidden size
        # output shape = (batch size, seq_len, 2 * hidden_size + num features)

        # linear layer
        self.linear_layer = nn.Linear(2 * hidden_size + num_feat,hidden_size_linear)
        # output size = (batch_size, seq_len, hidden_size_linear)

        # tanh - no change in dims
        self.tanh_layer = nn.Tanh()

        # dropout - no change of dimensions
        self.dropout_layer = nn.Dropout(dropout_keep)

        # fc layer - same operation as linear layer
        self.fc_layer = nn.Linear(hidden_size_linear,output_size)
        # output shape - (batch_size, hidden_size_linear)

        # softmax - no change
        self.softmax_layer = nn.Softmax(dim=0)
    
    def forward(self, x):
        # input shape = (batch size, sequence length, num features) | N,L,Hin
        lstm_output,_ = self.lstm_layer(a_inp)
        # output shape = (batch size, sequence length, 2 * hidden_size)

        # concatenate over num features and hidden size
        input_features = torch.cat([lstm_output,a_inp],2)
        # output shape = (batch size, sequence length, 2 * hidden_size + num features)

        # linear output
        linear_output = self.linear_layer(input_features)
        # output size = (batch_size, seq_len, hidden_size_linear)

        linear_output = linear_output.permute(0,2,1) # Reshaping fot max_pool

        max_out_features = F.max_pool1d(linear_output, linear_output.shape[2]).squeeze(2)
        # max_out_features.shape = (batch_size, hidden_size_linear)

        max_out_features = self.dropout_layer(max_out_features)
        final_out = self.fc_layer(max_out_features)
        res = self.softmax_layer(final_out)
        
        return res

In [172]:
test_loss = 0
train_loss = 0

# init
model = RCNN()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
loss = torch.nn.CrossEntropyLoss()

# one iteration of forward prop
optimizer.zero_grad()
model_out = model(a_inp)
fit = loss(model_out,a_out)

# backprop
fit.backward()
train_loss += fit.item()
optimizer.step()

# test loss calculation
with torch.no_grad():
    predicted_out = model(a_inp)
    fit = loss(mpredicted_outodel_out,a_out)
    test_loss += fit.item()

In [ ]:
# 64, 1024

# linear. - 1024,256

In [184]:
inp = torch.randn(64, 1024)
lin = nn.Linear(1024,256)

out,_ = lin(inp)

tensor([[-0.7925, -0.5907,  0.5451,  ..., -0.6407,  0.4399, -0.0260],
        [-0.4915,  0.3601, -0.6773,  ...,  0.1197, -0.3389, -0.5060],
        [-0.5017, -0.5919,  0.2340,  ...,  0.2594,  0.0522,  0.1281],
        ...,
        [-0.8002, -0.0891, -0.4553,  ...,  0.2615,  0.5677, -0.1631],
        [-0.2520, -0.0465, -1.1844,  ...,  0.0924, -0.9306,  0.2858],
        [ 0.1919,  0.4386, -0.3073,  ..., -0.3545, -1.2773, -0.1907]],
       grad_fn=<AddmmBackward0>)

In [185]:
lin

Linear(in_features=1024, out_features=256, bias=True)